# 🏍️ Prebuilt Motorcycles - Ithaka Powertrain Simulator

Welcome to the **Prebuilt Motorcycles Interface** - your gateway to testing professionally configured motorcycle powertrains.

**What this notebook offers:**
- 🏍️ **Select from expert-designed motorcycles** with realistic specifications
- 🗺️ **Test on real-world GPS tracks** for authentic performance analysis
- 🚀 **One-click physics simulations** with detailed results
- 📊 **Professional visualizations** and comprehensive metrics
- 💾 **Export-ready data** for engineering analysis

**Perfect for:**
- Quick performance comparisons between different motorcycle types
- Educational demonstrations of powertrain concepts
- Baseline testing before custom modifications
- CEO/stakeholder presentations with minimal setup

**No coding required** - simply run each cell and use the interactive controls.

In [ ]:
#@title 🚀 Environment Setup { display-mode: "form" }

#@markdown This cell automatically detects your environment and sets up all required dependencies.
#@markdown **Click the play button** to run this cell - setup takes about 30-60 seconds.

import sys
import os
import warnings
import subprocess
warnings.filterwarnings('ignore')

def setup_environment():
    """Smart environment setup for both Colab and local environments."""
    
    # Detect environment
    try:
        import google.colab
        IN_COLAB = True
        print("📦 Google Colab environment detected")
    except ImportError:
        IN_COLAB = False
        print("💻 Local environment detected")
    
    if IN_COLAB:
        # Check for existing repository
        print("🔍 Checking for repository...")
        
        current_dir = os.getcwd()
        possible_paths = ['/content/Teo-Moto-Repo', '/content/ithaka-powertrain-sim', current_dir]
        
        repo_found = False
        for path in possible_paths:
            if os.path.exists(path) and os.path.exists(os.path.join(path, 'setup.py')):
                print(f"✅ Found repository at: {path}")
                os.chdir(path)
                repo_found = True
                break
        
        if not repo_found:
            print("📥 Cloning repository...")
            try:
                result = subprocess.run(['git', 'clone', 'https://github.com/Teodus/Teo-Moto-Repo.git'], 
                                      capture_output=True, text=True, cwd='/content')
                if result.returncode == 0:
                    os.chdir('/content/Teo-Moto-Repo')
                    print("✅ Repository cloned successfully")
                else:
                    print("❌ Clone failed. Please check your internet connection.")
                    return False
            except Exception as e:
                print(f"❌ Error: {e}")
                return False
        
        # Install dependencies
        print("📦 Installing dependencies...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', 'requirements.txt'], 
                      capture_output=True)
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-e', '.'], 
                      capture_output=True)
        print("✅ Dependencies installed")
    
    # Verify installation
    try:
        print("🔍 Verifying installation...")
        import pandas as pd
        import numpy as np
        import matplotlib.pyplot as plt
        import ipywidgets as widgets
        from IPython.display import display, clear_output, HTML
        import json
        import glob
        from datetime import datetime
        
        # Import project modules
        from ithaka_powertrain_sim.component_library import (
            get_motorcycle_list, get_motorcycle_specs, create_motorcycle
        )
        from ithaka_powertrain_sim.motorbike import Motorbike
        from ithaka_powertrain_sim.trajectory import load_gpx, append_and_resample_dataframe
        
        print("✅ All modules imported successfully")
        print(f"📊 Found {len(get_motorcycle_list())} predefined motorcycles")
        
        gpx_files = glob.glob('docs/gpx_files/*.gpx')
        if gpx_files:
            print(f"🗺️ Found {len(gpx_files)} test tracks")
        
        return True
        
    except ImportError as e:
        print(f"❌ Import failed: {e}")
        if 'shortuuid' in str(e):
            print("💡 Installing missing dependency...")
            subprocess.run([sys.executable, '-m', 'pip', 'install', 'shortuuid'], 
                         capture_output=True)
            print("✅ Please re-run this cell")
        return False

# Run setup
print("🚀 Starting environment setup...")
setup_success = setup_environment()

if setup_success:
    print("\n🎉 Setup complete! Ready to test prebuilt motorcycles!")
    print("👇 Continue to Step 1 below")
else:
    print("\n❌ Setup failed. Please check the error messages above.")

In [ ]:
#@title 🏗️ Initialize Application { display-mode: "form" }

#@markdown This cell sets up the application for prebuilt motorcycles testing.
#@markdown **Run this cell** to prepare the simulation environment.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import json
import glob
import os
from datetime import datetime

# Import project modules
from ithaka_powertrain_sim.component_library import (
    get_motorcycle_list, get_motorcycle_specs, create_motorcycle
)
from ithaka_powertrain_sim.motorbike import Motorbike
from ithaka_powertrain_sim.trajectory import load_gpx, append_and_resample_dataframe

class PrebuiltMotorcycleApp:
    """Application state manager for prebuilt motorcycles."""
    
    def __init__(self):
        self.selected_motorcycle = None
        self.selected_tracks = []
        self.simulation_results = {}
        
    def get_available_tracks(self):
        """Get list of available GPX track files."""
        gpx_files = glob.glob('docs/gpx_files/*.gpx')
        if not gpx_files:
            return [("No tracks found", None)]
        
        track_names = []
        for gpx_file in gpx_files:
            name = os.path.basename(gpx_file).replace('.gpx', '').replace('-', ' ').replace('_', ' ')
            name = ' '.join(word.capitalize() for word in name.split())
            track_names.append((name, gpx_file))
        
        return sorted(track_names)
    
    def simulate_motorcycle_on_track(self, motorcycle, track_file, track_name):
        """Run simulation for motorcycle on single track."""
        try:
            # Load trajectory
            trajectory_df = load_gpx(track_file)
            trajectory_df = append_and_resample_dataframe(trajectory_df)
            
            # Extract parameters
            target_speed = trajectory_df["Target Speed"].to_list()
            delta_distance = np.diff(trajectory_df["Distance"], prepend=0).tolist()
            delta_elevation = np.diff(trajectory_df["Elevation"], prepend=0).tolist()
            approximate_time = trajectory_df["Target Time"].to_list()
            
            # Run simulation
            achieved_speeds = [trajectory_df["Target Speed"].iloc[0]]
            reporting_dataframe_rows = []
            
            for index in range(1, len(trajectory_df)):
                delta_time = approximate_time[index] - approximate_time[index - 1]
                
                achieved_speed, reporting_dataframe_row = motorcycle.calculate_achieved_speed(
                    achieved_speeds[index - 1], target_speed[index], delta_time,
                    delta_distance[index], delta_elevation[index]
                )
                
                achieved_speeds.append(achieved_speed)
                reporting_dataframe_rows.append(reporting_dataframe_row)
                
                # Adjust timing
                if index < len(trajectory_df) - 1 and achieved_speed != 0 and target_speed[index] != 0:
                    future_delta_time = (delta_distance[index + 1] / achieved_speed) - (
                        delta_distance[index + 1] / target_speed[index]
                    )
                    for j in range(index + 1, len(approximate_time)):
                        approximate_time[j] += future_delta_time
            
            # Combine results
            motorbike_dataframe = trajectory_df.copy()
            motorbike_dataframe["Approximate Time"] = approximate_time
            motorbike_dataframe["Achieved Speed"] = achieved_speeds
            
            if reporting_dataframe_rows:
                reporting_dataframe = pd.concat(reporting_dataframe_rows, ignore_index=True)
                final_results = motorbike_dataframe.join(reporting_dataframe)
            else:
                final_results = motorbike_dataframe
            
            # Calculate metrics
            total_time = approximate_time[-1] if approximate_time else 0
            total_distance = trajectory_df['Distance'].iloc[-1] if len(trajectory_df) > 0 else 0
            avg_speed = (total_distance / total_time * 3.6) if total_time > 0 else 0
            
            energy_consumed = 0
            if reporting_dataframe_rows and 'Energy Consumed (J)' in reporting_dataframe.columns:
                energy_consumed = reporting_dataframe['Energy Consumed (J)'].sum()
            
            return {
                'track_name': track_name,
                'track_file': track_file,
                'total_time_s': total_time,
                'total_distance_km': total_distance / 1000,
                'average_speed_kmh': avg_speed,
                'energy_consumed_kWh': energy_consumed / 3.6e6,
                'simulation_results': final_results,
                'trajectory': trajectory_df,
                'success': True
            }
            
        except Exception as e:
            return {
                'track_name': track_name,
                'track_file': track_file,
                'error': str(e),
                'success': False
            }

# Initialize application
app = PrebuiltMotorcycleApp()

# Define widget styles
style = {'description_width': '150px'}
layout_wide = widgets.Layout(width='500px')

print("✅ Prebuilt Motorcycles App initialized successfully!")
print("👇 Continue to Step 1 below")

## Step 1: Select Your Motorcycle

Choose from our library of professionally configured motorcycles, each with realistic specifications and performance characteristics.

In [ ]:
#@title 🏍️ Motorcycle Selection { display-mode: "form" }

#@markdown Browse and select from our library of prebuilt motorcycles.
#@markdown Each motorcycle has been professionally configured with realistic specifications.

# Load available motorcycles
try:
    available_motorcycles = get_motorcycle_list()
    if not available_motorcycles:
        print("❌ No motorcycles found in the library")
    else:
        print(f"📊 Found {len(available_motorcycles)} prebuilt motorcycles")
except Exception as e:
    print(f"❌ Error loading motorcycles: {e}")
    available_motorcycles = []

# Create motorcycle selector with enhanced options
if available_motorcycles:
    options = [("Select a motorcycle...", None)]
    for name in available_motorcycles:
        try:
            specs = get_motorcycle_specs(name)
            motorcycle_type = specs.get('type', 'Unknown')
            
            # Add power info if available
            power_info = ""
            if 'displacement_cc' in specs:
                power_info = f" - {specs['displacement_cc']}cc"
            elif 'battery_kWh' in specs:
                power_info = f" - {specs['battery_kWh']}kWh"
            
            display_name = f"{name} ({motorcycle_type}{power_info})"
            options.append((display_name, name))
        except:
            options.append((name, name))
    
    motorcycle_selector = widgets.Dropdown(
        options=options,
        value=None,
        description='Motorcycle:',
        style=style,
        layout=layout_wide
    )
else:
    motorcycle_selector = widgets.Dropdown(
        options=[("No motorcycles available", None)],
        value=None,
        description='Motorcycle:',
        style=style,
        layout=layout_wide
    )

motorcycle_info = widgets.Output()
confirm_selection_btn = widgets.Button(
    description='✅ Select This Motorcycle',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

def display_motorcycle_info(change):
    """Display detailed information about selected motorcycle."""
    if change.new is None:
        return
    
    with motorcycle_info:
        clear_output()
        try:
            specs = get_motorcycle_specs(change.new)
            bike = create_motorcycle(change.new)
            
            print(f"📋 {change.new} - Detailed Specifications")
            print("=" * 50)
            
            # Basic info
            print(f"🏷️ Type: {specs.get('type', 'Unknown')}")
            print(f"⚖️ Total Mass: {bike.mass:.1f} kg")
            print(f"🏗️ Base Mass: {bike.dry_mass_excluding_components:.1f} kg")
            
            # Power analysis
            total_power = 0
            power_components = []
            for component in bike.child_components:
                if hasattr(component, '_maximum_power_generation') and component._maximum_power_generation > 0:
                    power_kw = component._maximum_power_generation / 1000
                    total_power += component._maximum_power_generation
                    power_components.append(f"{component.name}: {power_kw:.1f} kW")
            
            if total_power > 0:
                print(f"⚡ Total Power: {total_power / 1000:.1f} kW")
                power_to_weight = (total_power / 1000) / bike.mass
                print(f"🏁 Power-to-Weight: {power_to_weight:.3f} kW/kg")
                
                print("\n🔧 Power Sources:")
                for comp_info in power_components:
                    print(f"  • {comp_info}")
            
            # Energy storage
            energy_components = []
            for component in bike.child_components:
                if hasattr(component, 'remaining_energy_capacity'):
                    energy_kwh = component.remaining_energy_capacity / 3.6e6
                    energy_components.append(f"{component.name}: {energy_kwh:.1f} kWh")
            
            if energy_components:
                print("\n🔋 Energy Storage:")
                for comp_info in energy_components:
                    print(f"  • {comp_info}")
            
            # Additional specs from database
            additional_specs = []
            if 'displacement_cc' in specs:
                additional_specs.append(f"Engine Displacement: {specs['displacement_cc']} cc")
            if 'fuel_L' in specs:
                additional_specs.append(f"Fuel Capacity: {specs['fuel_L']} L")
            if 'range_km' in specs:
                additional_specs.append(f"Estimated Range: {specs['range_km']} km")
            
            if additional_specs:
                print("\n📊 Additional Specifications:")
                for spec in additional_specs:
                    print(f"  • {spec}")
            
            # Component count
            print(f"\n🔧 Total Components: {len(bike.child_components)}")
            
            # Description
            description = specs.get('description', 'No description available')
            print(f"\n📝 Description:\n{description}")
            
        except Exception as e:
            print(f"❌ Error loading motorcycle info: {e}")

def confirm_motorcycle_selection(btn):
    """Confirm selection of motorcycle."""
    if motorcycle_selector.value:
        try:
            app.selected_motorcycle = create_motorcycle(motorcycle_selector.value)
            with motorcycle_info:
                clear_output()
                print(f"✅ {motorcycle_selector.value} selected successfully!")
                print(f"🏍️ Motorcycle: {app.selected_motorcycle.name}")
                print(f"⚖️ Total Mass: {app.selected_motorcycle.mass:.1f} kg")
                print("\n👇 Continue to Step 2 to choose test tracks")
        except Exception as e:
            with motorcycle_info:
                clear_output()
                print(f"❌ Error selecting motorcycle: {e}")
    else:
        with motorcycle_info:
            clear_output()
            print("❌ Please select a motorcycle first")

# Connect event handlers
motorcycle_selector.observe(display_motorcycle_info, names='value')
confirm_selection_btn.on_click(confirm_motorcycle_selection)

# Display interface
display(motorcycle_selector)
display(motorcycle_info)
display(confirm_selection_btn)

## Step 2: Choose Test Tracks

Select which real-world GPS tracks to test your motorcycle on for performance analysis.

In [ ]:
#@title 🗺️ Track Selection { display-mode: "form" }

#@markdown Select which tracks to test your motorcycle on:
#@markdown - **Single Track**: Test on one specific track for focused analysis
#@markdown - **All Tracks**: Test on all available tracks for comprehensive comparison

track_mode_selector = widgets.RadioButtons(
    options=[
        ('🎯 Test Single Track', 'single'),
        ('🌍 Test All Available Tracks', 'all')
    ],
    value=None,
    description='Test Mode:',
    style=style,
    layout=layout_wide
)

single_track_selector = widgets.Dropdown(
    options=[("Select a track...", None)],
    value=None,
    description='Track:',
    style=style,
    layout=layout_wide
)

track_info = widgets.Output()
confirm_tracks_btn = widgets.Button(
    description='✅ Confirm Track Selection',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

def update_track_options():
    """Update track dropdown with available tracks."""
    try:
        available_tracks = app.get_available_tracks()
        if available_tracks and available_tracks[0][1] is not None:
            options = [("Select a track...", None)] + available_tracks
            single_track_selector.options = options
        else:
            single_track_selector.options = [("No tracks available", None)]
    except Exception as e:
        print(f"Error loading tracks: {e}")

def handle_track_mode_change(change):
    """Handle track mode selection."""
    with track_info:
        clear_output()
        if change.new == 'single':
            update_track_options()
            single_track_selector.layout.display = 'block'
            print("📍 Select a single track for focused testing")
            print("💡 Single track mode provides detailed analysis of specific terrain")
        elif change.new == 'all':
            single_track_selector.layout.display = 'none'
            available_tracks = app.get_available_tracks()
            if available_tracks and available_tracks[0][1] is not None:
                print(f"🌍 Will test on all {len(available_tracks)} available tracks")
                print("\n📊 Track Overview:")
                for i, (name, _) in enumerate(available_tracks[:8]):  # Show first 8
                    print(f"  {i+1:2d}. {name}")
                if len(available_tracks) > 8:
                    print(f"  ... and {len(available_tracks) - 8} more")
                print("\n💡 All-tracks mode provides comprehensive performance comparison")
            else:
                print("❌ No tracks found in the repository")

def handle_single_track_change(change):
    """Handle single track selection."""
    if change.new and track_mode_selector.value == 'single':
        with track_info:
            clear_output()
            track_name = change.new[0] if isinstance(change.new, tuple) else str(change.new)
            print(f"📍 Selected Track: {track_name}")
            print("💡 This track will be used for detailed performance analysis")

def confirm_track_selection(btn):
    """Confirm track selection."""
    if not app.selected_motorcycle:
        with track_info:
            clear_output()
            print("❌ Please select a motorcycle first (Step 1)")
        return
    
    if not track_mode_selector.value:
        with track_info:
            clear_output()
            print("❌ Please select a test mode")
        return
    
    if track_mode_selector.value == 'single' and not single_track_selector.value:
        with track_info:
            clear_output()
            print("❌ Please select a track")
        return
    
    # Store selection
    if track_mode_selector.value == 'single':
        app.selected_tracks = [single_track_selector.value]
    else:
        available_tracks = app.get_available_tracks()
        app.selected_tracks = [(name, path) for name, path in available_tracks if path is not None]
    
    with track_info:
        clear_output()
        if track_mode_selector.value == 'single':
            track_name = app.selected_tracks[0][0] if isinstance(app.selected_tracks[0], tuple) else str(app.selected_tracks[0])
            print(f"✅ Selected Track: {track_name}")
            print(f"🏍️ Motorcycle: {app.selected_motorcycle.name}")
            print("📊 Ready for focused track analysis")
        else:
            print(f"✅ All {len(app.selected_tracks)} tracks selected")
            print(f"🏍️ Motorcycle: {app.selected_motorcycle.name}")
            print("📊 Ready for comprehensive performance comparison")
        print("\n👇 Continue to Step 3 to run simulation")

# Connect event handlers
track_mode_selector.observe(handle_track_mode_change, names='value')
single_track_selector.observe(handle_single_track_change, names='value')
confirm_tracks_btn.on_click(confirm_track_selection)

# Hide single track selector initially
single_track_selector.layout.display = 'none'

# Check if we have a motorcycle selected
if app.selected_motorcycle:
    display(track_mode_selector)
    display(single_track_selector)
    display(track_info)
    display(confirm_tracks_btn)
else:
    print("⚠️ Please complete Step 1 (select a motorcycle) first")

## Step 3: Run Physics Simulation

Execute the comprehensive powertrain simulation with real-world physics modeling.

In [ ]:
#@title 🚀 Run Physics Simulation { display-mode: "form" }

#@markdown Click the button below to start the physics-based simulation.
#@markdown 
#@markdown **The simulation calculates:**
#@markdown - Achieved speeds vs target speeds on real terrain
#@markdown - Energy consumption and efficiency
#@markdown - Component utilization and performance limits
#@markdown - Acceleration and braking capabilities

run_simulation_btn = widgets.Button(
    description='🚀 Run Simulation',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

simulation_progress = widgets.Output()

def run_simulation(btn):
    """Execute simulation for selected motorcycle and tracks."""
    # Check prerequisites
    if not app.selected_motorcycle:
        with simulation_progress:
            clear_output()
            print("❌ No motorcycle selected. Complete Step 1 first.")
        return
    
    if not app.selected_tracks:
        with simulation_progress:
            clear_output()
            print("❌ No tracks selected. Complete Step 2 first.")
        return
    
    with simulation_progress:
        clear_output()
        print("🚀 Starting Physics-Based Simulation")
        print("=" * 50)
        print(f"🏍️ Motorcycle: {app.selected_motorcycle.name}")
        print(f"⚖️ Total Mass: {app.selected_motorcycle.mass:.1f} kg")
        print(f"📊 Tracks to test: {len(app.selected_tracks)}")
        print("\n🔄 Running simulations...")
        
        app.simulation_results = {}
        successful_runs = 0
        total_distance = 0
        total_time = 0
        total_energy = 0
        
        for i, (track_name, track_file) in enumerate(app.selected_tracks):
            if track_file is None:
                continue
            
            print(f"\n📍 ({i+1}/{len(app.selected_tracks)}) Testing: {track_name}")
            
            try:
                result = app.simulate_motorcycle_on_track(app.selected_motorcycle, track_file, track_name)
                app.simulation_results[track_name] = result
                
                if result['success']:
                    successful_runs += 1
                    distance = result['total_distance_km']
                    time_min = result['total_time_s'] / 60
                    speed = result['average_speed_kmh']
                    energy = result['energy_consumed_kWh']
                    
                    total_distance += distance
                    total_time += result['total_time_s']
                    total_energy += energy
                    
                    print(f"  ✅ Completed - {time_min:.1f} min | {distance:.1f} km | {speed:.1f} km/h")
                    if energy > 0:
                        efficiency = energy / distance if distance > 0 else 0
                        print(f"     Energy: {energy:.2f} kWh | Efficiency: {efficiency:.3f} kWh/km")
                else:
                    error_msg = result.get('error', 'Unknown error')
                    print(f"  ❌ Failed: {error_msg[:60]}..." if len(error_msg) > 60 else f"  ❌ Failed: {error_msg}")
            
            except Exception as e:
                print(f"  ❌ Exception: {str(e)[:60]}..." if len(str(e)) > 60 else f"  ❌ Exception: {str(e)}")
                app.simulation_results[track_name] = {
                    'track_name': track_name,
                    'error': str(e),
                    'success': False
                }
        
        print("\n" + "=" * 50)
        print("🎉 SIMULATION COMPLETE!")
        print(f"✅ Successful: {successful_runs}/{len(app.selected_tracks)} tracks")
        
        if successful_runs > 0:
            avg_speed = (total_distance / (total_time / 3600)) if total_time > 0 else 0
            print(f"\n📊 Overall Performance Summary:")
            print(f"  🛣️ Total Distance: {total_distance:.1f} km")
            print(f"  ⏱️ Total Time: {total_time/3600:.1f} hours")
            print(f"  🏁 Average Speed: {avg_speed:.1f} km/h")
            if total_energy > 0:
                print(f"  🔋 Total Energy: {total_energy:.2f} kWh")
                print(f"  ⚡ Efficiency: {total_energy/total_distance:.3f} kWh/km")
            
            print("\n👇 Continue to Step 4 to view detailed results and export data")
        else:
            print("\n❌ No successful simulations completed.")
            print("💡 Check track files and motorcycle configuration.")

run_simulation_btn.on_click(run_simulation)

# Only show if prerequisites are met
if app.selected_motorcycle and app.selected_tracks:
    display(run_simulation_btn)
    display(simulation_progress)
else:
    missing = []
    if not app.selected_motorcycle:
        missing.append("motorcycle (Step 1)")
    if not app.selected_tracks:
        missing.append("tracks (Step 2)")
    
    print(f"⚠️ Complete the following first: {', '.join(missing)}")

## Step 4: Analyze Results & Export Data

View comprehensive simulation results with professional analysis and export capabilities.

In [ ]:
#@title 📊 Results Analysis & Export { display-mode: "form" }

#@markdown This section provides detailed analysis of simulation results and professional export options.
#@markdown **Use the buttons below** to view comprehensive results and export engineering-grade data.

view_results_btn = widgets.Button(
    description='📊 View Detailed Results',
    button_style='info',
    layout=widgets.Layout(width='180px')
)

export_results_btn = widgets.Button(
    description='💾 Export Complete Data',
    button_style='success',
    layout=widgets.Layout(width='180px')
)

results_display = widgets.Output()
export_status = widgets.Output()

def view_detailed_results(btn):
    """Display comprehensive simulation results with professional analysis."""
    if not app.simulation_results:
        with results_display:
            clear_output()
            print("❌ No simulation results available. Run simulation first.")
        return
    
    with results_display:
        clear_output()
        
        print("📊 COMPREHENSIVE SIMULATION ANALYSIS")
        print("=" * 70)
        
        # Motorcycle specifications summary
        print(f"🏍️ MOTORCYCLE: {app.selected_motorcycle.name}")
        print(f"⚖️ Total Mass: {app.selected_motorcycle.mass:.1f} kg")
        
        # Calculate and display power information
        total_power = 0
        power_sources = []
        energy_sources = []
        
        for component in app.selected_motorcycle.child_components:
            if hasattr(component, '_maximum_power_generation') and component._maximum_power_generation > 0:
                power_kw = component._maximum_power_generation / 1000
                total_power += component._maximum_power_generation
                power_sources.append(f"{component.name} ({power_kw:.1f} kW)")
            
            if hasattr(component, 'remaining_energy_capacity'):
                energy_kwh = component.remaining_energy_capacity / 3.6e6
                energy_sources.append(f"{component.name} ({energy_kwh:.1f} kWh)")
        
        if total_power > 0:
            power_to_weight = (total_power / 1000) / app.selected_motorcycle.mass
            print(f"⚡ Total Power: {total_power / 1000:.1f} kW")
            print(f"🏁 Power-to-Weight: {power_to_weight:.3f} kW/kg")
            print(f"🔧 Power Sources: {', '.join(power_sources)}")
        
        if energy_sources:
            print(f"🔋 Energy Sources: {', '.join(energy_sources)}")
        
        print("\n" + "=" * 70)
        
        # Detailed results table
        print("📈 TRACK-BY-TRACK PERFORMANCE RESULTS")
        print("-" * 95)
        print(f"{'Track Name':<35} {'Distance':<10} {'Time':<8} {'Speed':<10} {'Energy':<10} {'Efficiency':<12}")
        print(f"{'':35} {'(km)':<10} {'(min)':<8} {'(km/h)':<10} {'(kWh)':<10} {'(kWh/km)':<12}")
        print("-" * 95)
        
        successful_results = []
        failed_count = 0
        
        for track_name, result in app.simulation_results.items():
            if result.get('success'):
                dist = result['total_distance_km']
                time_min = result['total_time_s'] / 60
                speed = result['average_speed_kmh']
                energy = result['energy_consumed_kWh']
                efficiency = energy / dist if dist > 0 and energy > 0 else 0
                
                print(f"{track_name[:34]:<35} {dist:<10.1f} {time_min:<8.1f} {speed:<10.1f} {energy:<10.2f} {efficiency:<12.3f}")
                successful_results.append(result)
            else:
                failed_count += 1
                error_msg = result.get('error', 'Unknown error')[:20]
                print(f"{track_name[:34]:<35} {'ERROR':<10} {'---':<8} {'---':<10} {'---':<10} {error_msg:<12}")
        
        print("-" * 95)
        
        # Statistical analysis
        if successful_results:
            distances = [r['total_distance_km'] for r in successful_results]
            speeds = [r['average_speed_kmh'] for r in successful_results]
            energies = [r['energy_consumed_kWh'] for r in successful_results]
            times = [r['total_time_s'] for r in successful_results]
            
            print(f"\n📊 STATISTICAL ANALYSIS")
            print(f"✅ Successful simulations: {len(successful_results)}/{len(app.simulation_results)}")
            if failed_count > 0:
                print(f"❌ Failed simulations: {failed_count}")
            
            print(f"\n🏁 Speed Performance:")
            print(f"  • Average Speed: {np.mean(speeds):.1f} km/h")
            print(f"  • Max Speed: {np.max(speeds):.1f} km/h")
            print(f"  • Min Speed: {np.min(speeds):.1f} km/h")
            print(f"  • Speed Std Dev: {np.std(speeds):.1f} km/h")
            
            print(f"\n🛣️ Distance & Time:")
            print(f"  • Total Distance: {sum(distances):.1f} km")
            print(f"  • Total Time: {sum(times)/3600:.1f} hours")
            print(f"  • Average Track Length: {np.mean(distances):.1f} km")
            
            if any(e > 0 for e in energies):
                valid_energies = [e for e in energies if e > 0]
                valid_distances = [distances[i] for i, e in enumerate(energies) if e > 0]
                efficiencies = [e/d for e, d in zip(valid_energies, valid_distances) if d > 0]
                
                print(f"\n🔋 Energy Performance:")
                print(f"  • Total Energy Used: {sum(valid_energies):.2f} kWh")
                print(f"  • Average Efficiency: {np.mean(efficiencies):.3f} kWh/km")
                print(f"  • Best Efficiency: {np.min(efficiencies):.3f} kWh/km")
                print(f"  • Worst Efficiency: {np.max(efficiencies):.3f} kWh/km")
                
                if np.mean(efficiencies) > 0:
                    range_per_kwh = 1 / np.mean(efficiencies)
                    print(f"  • Average Range/kWh: {range_per_kwh:.1f} km/kWh")
            
            print(f"\n🎯 Performance Rating:")
            if power_to_weight > 0.15:
                print(f"  • Power Rating: High Performance ({power_to_weight:.3f} kW/kg)")
            elif power_to_weight > 0.08:
                print(f"  • Power Rating: Moderate Performance ({power_to_weight:.3f} kW/kg)")
            else:
                print(f"  • Power Rating: Efficiency Focused ({power_to_weight:.3f} kW/kg)")
            
            avg_speed = np.mean(speeds)
            if avg_speed > 45:
                print(f"  • Speed Rating: High Speed Capable ({avg_speed:.1f} km/h avg)")
            elif avg_speed > 35:
                print(f"  • Speed Rating: Moderate Speed ({avg_speed:.1f} km/h avg)")
            else:
                print(f"  • Speed Rating: Urban/Low Speed ({avg_speed:.1f} km/h avg)")
        
        else:
            print("\n❌ No successful results to analyze")

def export_simulation_results(btn):
    """Export comprehensive simulation results with full engineering data."""
    if not app.simulation_results:
        with export_status:
            clear_output()
            print("❌ No results to export. Run simulation first.")
        return
    
    try:
        with export_status:
            clear_output()
            print("💾 Preparing comprehensive engineering export...")
        
        # Create comprehensive export data structure
        export_data = {
            'export_metadata': {
                'timestamp': datetime.now().isoformat(),
                'notebook_type': 'prebuilt_motorcycles',
                'application_version': '2.0',
                'total_tracks_tested': len(app.simulation_results),
                'successful_simulations': len([r for r in app.simulation_results.values() if r.get('success')])
            },
            'motorcycle_specifications': {
                'name': app.selected_motorcycle.name,
                'total_mass_kg': float(app.selected_motorcycle.mass),
                'dry_mass_excluding_components_kg': float(app.selected_motorcycle.dry_mass_excluding_components),
                'front_mass_ratio': float(app.selected_motorcycle.front_mass_ratio),
                'wheelbase_m': float(app.selected_motorcycle.wheelbase),
                'front_wheel_radius_m': float(app.selected_motorcycle.front_wheel_radius),
                'rear_wheel_radius_m': float(app.selected_motorcycle.rear_wheel_radius),
                'frontal_area_m2': float(app.selected_motorcycle.frontal_area),
                'coefficient_of_aerodynamic_drag': float(app.selected_motorcycle.coefficient_of_aerodynamic_drag),
                'components': []
            },
            'simulation_results': {},
            'performance_analysis': {},
            'engineering_summary': {}
        }
        
        # Add detailed component specifications
        total_power = 0
        total_energy_capacity = 0
        
        for component in app.selected_motorcycle.child_components:
            comp_spec = {
                'name': component.name,
                'type': type(component).__name__,
                'mass_kg': float(component.mass)
            }
            
            # Power generation capability
            if hasattr(component, '_maximum_power_generation'):
                power_w = component._maximum_power_generation
                comp_spec['max_power_W'] = float(power_w)
                comp_spec['max_power_kW'] = float(power_w / 1000)
                total_power += power_w
            
            # Energy storage capability
            if hasattr(component, 'remaining_energy_capacity'):
                energy_j = component.remaining_energy_capacity
                comp_spec['energy_capacity_J'] = float(energy_j)
                comp_spec['energy_capacity_kWh'] = float(energy_j / 3.6e6)
                total_energy_capacity += energy_j
            
            # Efficiency characteristics
            if hasattr(component, 'efficiency_function'):
                comp_spec['has_efficiency_curve'] = True
            
            export_data['motorcycle_specifications']['components'].append(comp_spec)
        
        # Add calculated totals
        export_data['motorcycle_specifications']['total_power_kW'] = float(total_power / 1000)
        export_data['motorcycle_specifications']['total_energy_capacity_kWh'] = float(total_energy_capacity / 3.6e6)
        if app.selected_motorcycle.mass > 0:
            export_data['motorcycle_specifications']['power_to_weight_ratio_kW_per_kg'] = float((total_power / 1000) / app.selected_motorcycle.mass)
        
        # Add simulation results with full details
        successful_results = []
        for track_name, result in app.simulation_results.items():
            if result.get('success'):
                track_data = {
                    'track_name': result['track_name'],
                    'track_file': result['track_file'],
                    'total_time_s': float(result['total_time_s']),
                    'total_distance_km': float(result['total_distance_km']),
                    'average_speed_kmh': float(result['average_speed_kmh']),
                    'energy_consumed_kWh': float(result['energy_consumed_kWh']),
                    'status': 'success'
                }
                
                # Add efficiency metrics
                if result['total_distance_km'] > 0 and result['energy_consumed_kWh'] > 0:
                    track_data['energy_efficiency_kWh_per_km'] = float(result['energy_consumed_kWh'] / result['total_distance_km'])
                    track_data['range_per_kWh_km'] = float(result['total_distance_km'] / result['energy_consumed_kWh'])
                
                export_data['simulation_results'][track_name] = track_data
                successful_results.append(result)
            else:
                export_data['simulation_results'][track_name] = {
                    'track_name': result['track_name'],
                    'track_file': result.get('track_file', 'unknown'),
                    'error': result.get('error', 'Unknown error'),
                    'status': 'failed'
                }
        
        # Add performance analysis
        if successful_results:
            speeds = [r['average_speed_kmh'] for r in successful_results]
            distances = [r['total_distance_km'] for r in successful_results]
            energies = [r['energy_consumed_kWh'] for r in successful_results]
            times = [r['total_time_s'] for r in successful_results]
            
            export_data['performance_analysis'] = {
                'speed_statistics': {
                    'average_kmh': float(np.mean(speeds)),
                    'maximum_kmh': float(np.max(speeds)),
                    'minimum_kmh': float(np.min(speeds)),
                    'standard_deviation_kmh': float(np.std(speeds))
                },
                'distance_time_totals': {
                    'total_distance_km': float(sum(distances)),
                    'total_time_hours': float(sum(times) / 3600),
                    'average_track_length_km': float(np.mean(distances))
                },
                'energy_analysis': {
                    'total_energy_consumed_kWh': float(sum(energies)),
                    'average_efficiency_kWh_per_km': float(sum(energies) / sum(distances)) if sum(distances) > 0 else 0
                }
            }
            
            # Add energy efficiency statistics if available
            if any(e > 0 for e in energies):
                valid_energies = [e for e in energies if e > 0]
                valid_distances = [distances[i] for i, e in enumerate(energies) if e > 0]
                efficiencies = [e/d for e, d in zip(valid_energies, valid_distances) if d > 0]
                
                if efficiencies:
                    export_data['performance_analysis']['energy_analysis'].update({
                        'efficiency_statistics': {
                            'average_kWh_per_km': float(np.mean(efficiencies)),
                            'best_efficiency_kWh_per_km': float(np.min(efficiencies)),
                            'worst_efficiency_kWh_per_km': float(np.max(efficiencies)),
                            'efficiency_std_dev': float(np.std(efficiencies)),
                            'average_range_per_kWh_km': float(1 / np.mean(efficiencies)) if np.mean(efficiencies) > 0 else 0
                        }
                    })
        
        # Add engineering summary
        export_data['engineering_summary'] = {
            'test_completion_rate': len(successful_results) / len(app.simulation_results) if app.simulation_results else 0,
            'motorcycle_category': 'prebuilt',
            'primary_powertrain': 'hybrid' if total_power > 0 and total_energy_capacity > 0 else 'electric' if total_energy_capacity > 0 else 'ice',
            'performance_class': 'high' if (total_power / 1000) / app.selected_motorcycle.mass > 0.15 else 'moderate' if (total_power / 1000) / app.selected_motorcycle.mass > 0.08 else 'efficiency_focused'
        }
        
        # Generate filename with motorcycle name and timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        motorcycle_name = app.selected_motorcycle.name.replace(' ', '_').replace('/', '_').lower()
        filename = f"prebuilt_{motorcycle_name}_{timestamp}.json"
        
        # Write comprehensive data to file
        with open(filename, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        with export_status:
            clear_output()
            print("✅ EXPORT COMPLETED SUCCESSFULLY!")
            print("=" * 50)
            print(f"📄 Filename: {filename}")
            
            file_size = os.path.getsize(filename) / 1024  # KB
            print(f"📦 File Size: {file_size:.1f} KB")
            
            print("\n📊 Export Contains:")
            print(f"  🏍️ Complete motorcycle specifications ({len(export_data['motorcycle_specifications']['components'])} components)")
            print(f"  📈 {len(app.simulation_results)} track simulation results")
            print(f"  📊 Statistical performance analysis")
            print(f"  🔧 Engineering summary and classifications")
            print(f"  📋 Full metadata and traceability information")
            
            if successful_results:
                print(f"\n🎯 Key Metrics Exported:")
                avg_speed = np.mean([r['average_speed_kmh'] for r in successful_results])
                total_dist = sum([r['total_distance_km'] for r in successful_results])
                print(f"  • Average Speed: {avg_speed:.1f} km/h")
                print(f"  • Total Distance Tested: {total_dist:.1f} km")
                print(f"  • Power-to-Weight: {(total_power/1000)/app.selected_motorcycle.mass:.3f} kW/kg")
            
            print("\n🎉 Ready for engineering analysis and reporting!")
            
    except Exception as e:
        with export_status:
            clear_output()
            print(f"❌ Export failed: {str(e)}")
            print("💡 Please check file permissions and available disk space")

# Connect button handlers
view_results_btn.on_click(view_detailed_results)
export_results_btn.on_click(export_simulation_results)

# Only show if simulation has been run
if app.simulation_results:
    display(widgets.HBox([view_results_btn, export_results_btn]))
    display(results_display)
    display(export_status)
else:
    print("⚠️ Run simulation first (Step 3) to view and export results")

## 🎉 Analysis Complete!

### What You've Accomplished:

- **🏍️ Tested Professional Motorcycles**: Selected and analyzed expertly-configured motorcycle powertrains
- **🗺️ Real-World Performance**: Evaluated performance on authentic GPS tracks with real terrain challenges
- **🚀 Physics-Based Simulation**: Ran comprehensive simulations with accurate energy and performance modeling
- **📊 Professional Analysis**: Generated detailed performance metrics and engineering insights
- **💾 Engineering Export**: Created comprehensive JSON files with all specifications and results

### Key Advantages of Prebuilt Motorcycles:

- **⚡ Rapid Testing**: Quick selection and immediate simulation without component configuration
- **🎯 Proven Configurations**: Each motorcycle represents realistic, well-balanced designs
- **📈 Baseline Performance**: Perfect for establishing performance benchmarks
- **🎓 Educational Value**: Demonstrates various powertrain architectures and their trade-offs
- **📋 Comprehensive Data**: Full engineering specifications readily available

### Next Steps:

1. **🔄 Compare Different Models**: Test various prebuilt motorcycles to understand design trade-offs
2. **📊 Analyze Exported Data**: Use the JSON files for detailed engineering analysis
3. **🏁 Performance Optimization**: Identify which motorcycles excel in specific conditions
4. **📝 Generate Reports**: Results are ready for presentations and technical documentation
5. **🔧 Custom Modifications**: Use insights to inform custom motorcycle designs

The exported data contains everything needed for comprehensive powertrain analysis, including complete motorcycle specifications, component details, and performance metrics across all tested tracks.